<a href="https://colab.research.google.com/github/nchadburn/Stat598-FinalProject/blob/main/MPLA%20Holdings%20Service.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# finanance data
import yfinance as yf

#visualization
import matplotlib.pylab as plt
import matplotlib.dates as mdates
import seaborn as sns

#date manipulation
from datetime import datetime
import pandas as pd
import numpy as np

#Autoreg and Arima models
from statsmodels.tsa.ar_model import AutoReg
from statsmodels.tsa.arima.model import ARIMA

# statstical testing, plotting and decompositions
from statsmodels.tsa.stattools import adfuller,acf, pacf
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.metrics import mean_squared_error

In [37]:

url = "https://raw.githubusercontent.com/Aditya-Patel/Stat598-FinalProject/main/mlpa_full-holdings.csv"

df_holdings = pd.read_csv(url)
df_holdings = df_holdings[(df_holdings['Name'] != 'OTHER PAYABLE & RECEIVABLES') & (df_holdings['Name'] != 'CASH')]


# Displaying the first few rows of the updated dataframe

df_holdings[f'Market Value ($)'] = df_holdings[f'Market Value ($)'].str.replace(',', '').astype(float)

total_market_value = df_holdings[f'Market Value ($)'].sum()
df_holdings['Percentage Holdings By Value'] = (df_holdings[f'Market Value ($)'] / total_market_value)

df_holdings



,% of Net Assets,Ticker,Name,SEDOL,Market Price ($),Shares Held,Market Value ($),Percentage Holdings By Value
0,20.87,ET,ENERGY TRANSFER LP,B0WHXD2,13.33,"22,735,436.84",3.030634e+08,2.004323e-01
1,13.76,EPD,ENTERPRISE PRODU,2285388,26.25,"7,611,951.00",1.998137e+08,1.321477e-01
2,12.17,MPLX,MPLX LP,B847R56,35.87,"4,925,004.00",1.766599e+08,1.168348e-01
3,10.80,PAA,PLAINS ALL AMER,2311960,14.95,"10,486,658.00",1.567755e+08,1.036842e-01
4,10.32,WES,WESTERN MIDSTREA,BJ552C8,28.24,"5,303,028.00",1.497575e+08,9.904279e-02
5,9.47,ENLC,ENLINK MIDSTREAM LLC,BJTNDQ9,12.81,"10,731,120.00",1.374656e+08,9.091352e-02
6,8.26,CQP,CHENIERE ENERGY,B1VNH36,59.30,"2,023,409.00",1.199882e+08,7.935470e-02
7,6.53,NS,NUSTAR ENERGY LP,2738804,19.00,"4,991,913.00",9.484635e+07,6.272705e-02
8,5.91,HESM,HESS MIDSTREAM-A,BKPK695,30.70,"2,795,220.00",8.581325e+07,5.675298e-02
9,4.53,GEL,GENESIS ENERGY L.P.,2372257,12.32,"5,334,251.00",6.571797e+07,4.346287e-02


In [42]:

start_date = '2020-06-01'
end_date = '2023-12-01'

etf_ticker = 'MLPA'

all_stocks_data = yf.download(etf_ticker, start=start_date, end=end_date)
all_stocks_data['Ticker'] = etf_ticker
all_stocks_data.columns = [f'{etf_ticker}_{col}' if col != 'Ticker' else col for col in all_stocks_data.columns]

for ticker in df_holdings['Ticker']:
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    percentage_holding = df_holdings.loc[df_holdings['Ticker'] == ticker, 'Percentage Holdings By Value'].iloc[0]
    all_stocks_data[f'{ticker}_Percent_Holding'] = percentage_holding
    stock_data.columns = [f'{ticker}_{col}' if col != 'Ticker' else col for col in stock_data.columns]

    all_stocks_data = all_stocks_data.join(stock_data, how='outer')

all_stocks_data.fillna(0, inplace=True)
weighted_values = pd.Series(0, index=all_stocks_data.index)
sum_values = pd.Series(0, index=all_stocks_data.index)
for ticker in df_holdings['Ticker']:
    weighted_values += all_stocks_data[f'{ticker}_Close'] * all_stocks_data[f'{ticker}_Percent_Holding']
    sum_values += all_stocks_data[f'{ticker}_Close']


all_stocks_data['Weighted Stock Value'] = weighted_values
all_stocks_data['Sum Stock Value'] = sum_values

all_stocks_data.reset_index(inplace=True)
all_stocks_data.tail()



[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

,Date,MLPA_Open,MLPA_High,MLPA_Low,MLPA_Close,MLPA_Adj Close,MLPA_Volume,Ticker,ET_Percent_Holding,ET_Open,...,SNMP_Volume,USDP_Percent_Holding,USDP_Open,USDP_High,USDP_Low,USDP_Close,USDP_Adj Close,USDP_Volume,Weighted Stock Value,Sum Stock Value
878,2023-11-24,45.310001,45.450001,45.029999,45.200001,45.200001,32500,MLPA,0.200432,13.71,...,4700.0,1.488047e-08,0.250,0.280,0.225,0.225,0.225,72000,25.016806,529.494997
879,2023-11-27,45.250000,45.330002,45.009998,45.250000,45.250000,48900,MLPA,0.200432,13.71,...,7600.0,1.488047e-08,0.215,0.233,0.211,0.226,0.226,105500,24.955155,527.801000
880,2023-11-28,45.250000,45.259998,44.799999,45.250000,45.250000,91800,MLPA,0.200432,13.70,...,2000.0,1.488047e-08,0.229,0.270,0.225,0.225,0.225,375600,25.002750,528.994998
881,2023-11-29,45.320000,45.660000,45.090000,45.500000,45.500000,63700,MLPA,0.200432,13.75,...,4200.0,1.488047e-08,0.221,0.260,0.221,0.243,0.243,106300,25.287195,541.103003
882,2023-11-30,45.540001,46.119999,45.400002,46.119999,46.119999,73000,MLPA,0.200432,13.75,...,8300.0,1.488047e-08,0.240,0.273,0.240,0.260,0.260,69200,25.661618,553.160002


[*********************100%%**********************]  1 of 1 completed
<bound method NDFrame.head of              Open   High    Low  Close  Adj Close    Volume
Date                                                       
2022-01-03   8.27   8.75   8.19   8.71   7.332820  29422600
2022-01-04   8.80   9.05   8.76   8.92   7.509617  24559000
2022-01-05   9.03   9.09   8.80   8.81   7.417010  16002200
2022-01-06   9.08   9.11   8.81   9.08   7.644319  14648400
2022-01-07   9.10   9.31   9.06   9.24   7.779020  21862100
...           ...    ...    ...    ...        ...       ...
2023-11-24  13.71  13.83  13.71  13.72  13.720000   5206700
2023-11-27  13.71  13.78  13.55  13.67  13.670000  10304700
2023-11-28  13.70  13.77  13.61  13.67  13.670000   8347700
2023-11-29  13.75  13.75  13.62  13.68  13.680000  12242400
2023-11-30  13.75  13.91  13.69  13.89  13.890000  16928700

[481 rows x 6 columns]>
Index(['ET_Open', 'ET_High', 'ET_Low', 'ET_Close', 'ET_Adj Close',
       'ET_Volume'],
      dt

,Date,MLPA_Open,MLPA_High,MLPA_Low,MLPA_Close,MLPA_Adj Close,MLPA_Volume,Ticker,ET_Open,ET_High,...,SNMP_Low,SNMP_Close,SNMP_Adj Close,SNMP_Volume,USDP_Open,USDP_High,USDP_Low,USDP_Close,USDP_Adj Close,USDP_Volume
0,2022-01-03,35.160000,36.150002,35.160000,36.070000,31.624790,316900,MLPA,8.27,8.75,...,NaN,NaN,NaN,NaN,5.31,5.57,5.300,5.57,4.965909,46600
1,2022-01-04,36.330002,36.939999,36.330002,36.830002,32.291130,205000,MLPA,8.80,9.05,...,NaN,NaN,NaN,NaN,5.69,5.69,5.510,5.52,4.921332,40700
2,2022-01-05,37.000000,37.310001,36.430000,36.430000,31.940428,255700,MLPA,9.03,9.09,...,NaN,NaN,NaN,NaN,5.63,5.72,5.565,5.61,5.001572,70000
3,2022-01-06,36.900002,37.259998,36.520000,37.070000,32.501549,379400,MLPA,9.08,9.11,...,NaN,NaN,NaN,NaN,5.60,5.73,5.500,5.62,5.010487,83800
4,2022-01-07,37.119999,37.549999,37.070000,37.509998,32.887329,377700,MLPA,9.10,9.31,...,NaN,NaN,NaN,NaN,5.60,5.70,5.470,5.56,4.956994,51100


In [ ]:
split_percentage = .8


column_reference ="Close"
moving_average_list =[2,5,14]
adf_significance_level = .05
periods_to_test = [7, 14, 28,60, 90,180, 365]




all_stocks_data.reset_index(inplace=True)